<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/evaluation/argilla_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install argilla -U
%pip install datasets

In [2]:
import argilla as rg

rg.init(
    api_url="https://winterForestStump-thesis.hf.space",
    api_key="admin.apikey",
    workspace="admin")

In [3]:
from datasets import Dataset
import pandas as pd

# Adding datasets GeneralQA

In [ ]:
ds=pd.read_csv('https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_3M%20CO.json.csv')
data_set = Dataset.from_pandas(ds)
data_set

Dataset({
    features: ['Unnamed: 0', 'question', 'company', 'response', 'context', 'retrieval_grade', 'hallucination_grade', 'answer_grade'],
    num_rows: 35
})

In [ ]:
feedback_dataset = rg.FeedbackDataset(
    guidelines="Grade the retrieval and generation results of the RAG chain",
    fields=[
        rg.TextField(name="query", title="User's question (query)"),
        rg.TextField(name="retrieved_document", title="Retrieved by the RAG chain documents (context)"),
        rg.TextField(name="generated_response", title="Generated LLM response (answer) to the question"),
    ],
    questions=[
        rg.LabelQuestion(
            name="relevancy",
            title="Are the retrieved documents relevant to the given question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="faithfulness",
            title="Is the generated answer grounded in / supported by a context (retrieved documents)?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="usefulness",
            title="Is generated answer useful to resolve a question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
    ]
)
feedback_dataset

FeedbackDataset(
   fields=[TextField(name='query', title="User's question (query)", required=True, type='text', use_markdown=False), TextField(name='retrieved_document', title='Retrieved by the RAG chain documents (context)', required=True, type='text', use_markdown=False), TextField(name='generated_response', title='Generated LLM response (answer) to the question', required=True, type='text', use_markdown=False)]
   questions=[LabelQuestion(name='relevancy', title='Are the retrieved documents relevant to the given question?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None), LabelQuestion(name='faithfulness', title='Is the generated answer grounded in / supported by a context (retrieved documents)?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None), LabelQuestion(name='usefulness', title='Is generated answer useful to resolve a question?', description=None, requir

In [ ]:
records = []
for i, item in enumerate(data_set):
    records.append(
        rg.FeedbackRecord(
            fields={
                "query": item["question"],
                "retrieved_document": item['context'],
                "generated_response": item["response"],
            },
            external_id=f"record-{i}"
        )
    )

feedback_dataset.add_records(records)

In [ ]:
remote_dataset = feedback_dataset.push_to_argilla(name="my-dataset")

# Adding records to the dataset General QA

In [ ]:
dataset = rg.FeedbackDataset.from_argilla(name="my-dataset", workspace="admin")

In [ ]:
files = [
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_COCA%20COLA%20CO.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_AMAZON%20COM%20INC.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_JPMORGAN%20CHASE%20%26%20CO.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_LOCKHEED%20MARTIN%20CORP.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_MICROSOFT%20CORP.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_NIKE%2C%20Inc..json.csv",
         "https://github.com/winterForestStump/thesis/blob/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_PayPal%20Holdings%2C%20Inc..json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_VERIZON%20COMMUNICATIONS%20INC.json.csv",
         "https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_Walmart%20Inc..json.csv"
         ]

In [ ]:
records = []
length = 35
for file in files:
    try:
        # Attempt to read the CSV file
        ds = pd.read_csv(file)
        data_set = Dataset.from_pandas(ds)

        # Process each record in the dataset
        for i, item in enumerate(data_set):
            records.append(rg.FeedbackRecord(fields={"query": item["question"],
                                                     "retrieved_document": item['context'],
                                                     "generated_response": item["response"]
                                                     },
                                            ))
        length += len(data_set)

    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred with {file}: {e}")


# Output the results
print(f"Processed {len(records)} records.")

Error parsing https://github.com/winterForestStump/thesis/blob/main/evaluation/bge-reranker_x_phi3-4k/csv/eval_PayPal%20Holdings%2C%20Inc..json.csv: Error tokenizing data. C error: Expected 1 fields in line 40, saw 20

Processed 280 records.


In [ ]:
length

315

In [ ]:
dataset.add_records(records)

Output()

# Collect responses General QA

In [4]:
feedback = rg.FeedbackDataset.from_argilla("my-dataset", workspace="admin")

In [5]:
list_values = []
for record_ix,record in enumerate(feedback):
  list_values.append({"record": str(record_ix+1),
                      "id": feedback.records[record_ix].id,
                      "query": feedback.records[record_ix].fields['query'],
                      "retrieved_document": feedback.records[record_ix].fields['retrieved_document'],
                      "generated_response": feedback.records[record_ix].fields['generated_response'],
                      "relevancy_value": feedback.records[record_ix].responses[0].values["relevancy"].value,
                      "faithfulness_value": feedback.records[record_ix].responses[0].values["faithfulness"].value,
                      "usefulness_value": feedback.records[record_ix].responses[0].values["usefulness"].value
                      })

In [6]:
len(list_values)

347

In [7]:
df = pd.DataFrame(list_values)
df.to_csv('eval_resuts_argilla_generalQA.csv')

# Creating dataset Financebench150

In [8]:
ds=pd.read_csv('https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/financebench150/csv/eval_v2.json.csv')
data_set = Dataset.from_pandas(ds)
data_set

Dataset({
    features: ['Unnamed: 0', 'question', 'response', 'context', 'hallucination_grade', 'answer_grade', 'ground_truth', 'evidence'],
    num_rows: 150
})

In [9]:
feedback_dataset = rg.FeedbackDataset(
    guidelines="Grade the retrieval and generation results of the RAG chain",
    fields=[
        rg.TextField(name="query", title="User's question (query)"),
        rg.TextField(name="retrieved_document", title="Retrieved by the RAG chain documents (context)"),
        rg.TextField(name="generated_response", title="Generated LLM response (answer) to the question"),
        rg.TextField(name="ground_truth", title="Ground Truth (Correct Answer)"),
        rg.TextField(name="evidence", title="Evidence context for the ground truth answer")
    ],
    questions=[
        rg.LabelQuestion(
            name="relevancy",
            title="Are the retrieved documents relevant to the given question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="faithfulness",
            title="Is the generated answer grounded in / supported by a context (retrieved documents)?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="usefulness",
            title="Is generated answer useful to resolve a question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="evidence",
            title="Are the retrieved documents relevant to the provided evidence context?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="correctness",
            title="Does the generated answer match the ground truth answer?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
    ]
)
feedback_dataset

FeedbackDataset(
   fields=[TextField(name='query', title="User's question (query)", required=True, type='text', use_markdown=False), TextField(name='retrieved_document', title='Retrieved by the RAG chain documents (context)', required=True, type='text', use_markdown=False), TextField(name='generated_response', title='Generated LLM response (answer) to the question', required=True, type='text', use_markdown=False), TextField(name='ground_truth', title='Ground Truth (Correct Answer)', required=True, type='text', use_markdown=False), TextField(name='evidence', title='Evidence context for the ground truth answer', required=True, type='text', use_markdown=False)]
   questions=[LabelQuestion(name='relevancy', title='Are the retrieved documents relevant to the given question?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None), LabelQuestion(name='faithfulness', title='Is the generated answer grounded in / supported by a context (re

In [10]:
records = []
for i, item in enumerate(data_set):
    records.append(
        rg.FeedbackRecord(
            fields={
                "query": item["question"],
                "retrieved_document": item['context'],
                "generated_response": item["response"],
                "ground_truth": item["ground_truth"],
                "evidence": item["evidence"]
            },
            external_id=f"record-{i}"
        )
    )

feedback_dataset.add_records(records)

In [11]:
remote_dataset = feedback_dataset.push_to_argilla(name="financebench150")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=75b8c025-0fde-4b0a-b522-728c3319b631
   name=financebench150
   workspace=Workspace(id=2afdc5dd-7ac5-49bb-99c9-73816b3e107a, name=admin, inserted_at=2024-06-22 19:41:07.791321, updated_at=2024-06-22 19:41:07.791321)
   url=https://winterforeststump-thesis.hf.space/dataset/75b8c025-0fde-4b0a-b522-728c3319b631/annotation-mode
   fields=[RemoteTextField(id=UUID('ea0bbadf-3ac4-4b75-b2f4-61c05e2678e1'), client=None, name='query', title="User's question (query)", required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('38dee56f-8bc3-47ba-9716-d0865bc666c4'), client=None, name='retrieved_document', title='Retrieved by the RAG chain documents (context)', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('1640515a-6cd3-499e-90dc-836718b11cc2'), client=None, name='generated_response', title

# Collect responses Financebench150

In [4]:
feedback = rg.FeedbackDataset.from_argilla("financebench150", workspace="admin")

In [5]:
list_values = []
for record_ix,record in enumerate(feedback):
  list_values.append({"record": str(record_ix+1),
                      "id": feedback.records[record_ix].id,
                      "query": feedback.records[record_ix].fields['query'],
                      "retrieved_document": feedback.records[record_ix].fields['retrieved_document'],
                      "generated_response": feedback.records[record_ix].fields['generated_response'],
                      "relevancy_value": feedback.records[record_ix].responses[0].values["relevancy"].value,
                      "faithfulness_value": feedback.records[record_ix].responses[0].values["faithfulness"].value,
                      "usefulness_value": feedback.records[record_ix].responses[0].values["usefulness"].value,
                      "evidence_value": feedback.records[record_ix].responses[0].values["evidence"].value,
                      "correctness_value": feedback.records[record_ix].responses[0].values["correctness"].value
                      })

In [6]:
len(list_values)

150

In [7]:
df = pd.DataFrame(list_values)
df.to_csv('eval_resuts_argilla_financebench150.csv')

# Adding datasets NoRag RegQA

In [4]:
ds=pd.read_csv('https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/phi-3_x_no_rag/csv/eval_no_rag.json.csv')
data_set = Dataset.from_pandas(ds)
data_set

Dataset({
    features: ['Unnamed: 0', 'question', 'company', 'response'],
    num_rows: 350
})

In [5]:
feedback_dataset = rg.FeedbackDataset(
    guidelines="Grade the generated answers",
    fields=[
        rg.TextField(name="query", title="User's question (query)"),
        rg.TextField(name='company_name', title='The name of the company'),
        rg.TextField(name="generated_response", title="Generated LLM response (answer) to the question"),
    ],
    questions=[
        rg.LabelQuestion(
            name="faithfulness",
            title="Does the generated answer contain truthful information?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="usefulness",
            title="Is the generated answer useful to resolve a question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
    ]
)
feedback_dataset

FeedbackDataset(
   fields=[TextField(name='query', title="User's question (query)", required=True, type='text', use_markdown=False), TextField(name='company_name', title='The name of the company', required=True, type='text', use_markdown=False), TextField(name='generated_response', title='Generated LLM response (answer) to the question', required=True, type='text', use_markdown=False)]
   questions=[LabelQuestion(name='faithfulness', title='Does the generated answer contain truthful information?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None), LabelQuestion(name='usefulness', title='Is the generated answer useful to resolve a question?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None)]
   guidelines=Grade the generated answers)
   metadata_properties=[])
   vectors_settings=[])
)

In [7]:
records = []
for i, item in enumerate(data_set):
    records.append(
        rg.FeedbackRecord(
            fields={
                "query": item["question"],
                "company_name": item["company"],
                "generated_response": item["response"],
            },
            external_id=f"record-{i}"
        )
    )

feedback_dataset.add_records(records)

In [8]:
remote_dataset = feedback_dataset.push_to_argilla(name="noRag-generalQA")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=b70e2bd8-d2b6-40d0-9b9c-a24eea6bffd3
   name=noRag-generalQA
   workspace=Workspace(id=142a9c2d-f1dc-4704-8188-daf87f726164, name=admin, inserted_at=2024-07-03 09:01:38.246747, updated_at=2024-07-03 09:01:38.246747)
   url=https://winterforeststump-thesis.hf.space/dataset/b70e2bd8-d2b6-40d0-9b9c-a24eea6bffd3/annotation-mode
   fields=[RemoteTextField(id=UUID('a5cd7df7-418a-4b2f-b117-44b098af1603'), client=None, name='query', title="User's question (query)", required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('d3704a8d-dad8-45ea-80f0-b205c1e7146c'), client=None, name='company_name', title='The name of the company', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('6131943f-9174-4b8b-adde-cee1f9ae9ebc'), client=None, name='generated_response', title='Generated LLM response (ans

# Collect responses NO RAG Generaal QA

In [4]:
feedback = rg.FeedbackDataset.from_argilla("noRag-generalQA", workspace="admin")

In [5]:
list_values = []
for record_ix,record in enumerate(feedback):
  list_values.append({"record": str(record_ix+1),
                      "id": feedback.records[record_ix].id,
                      "query": feedback.records[record_ix].fields['query'],
                      "company_name": feedback.records[record_ix].fields['company_name'],
                      "generated_response": feedback.records[record_ix].fields['generated_response'],
                      "faithfulness_value": feedback.records[record_ix].responses[0].values["faithfulness"].value,
                      "usefulness_value": feedback.records[record_ix].responses[0].values["usefulness"].value,
                      })

In [6]:
len(list_values)

350

In [7]:
df = pd.DataFrame(list_values)
df.to_csv('eval_resuts_argilla_NoRagGeneralQA.csv')

# Adding datasets NoRag FinBench150

In [4]:
ds=pd.read_csv('https://raw.githubusercontent.com/winterForestStump/thesis/main/evaluation/phi-3_x_no_rag/csv/financebench_eval_no_rag.json.csv')
data_set = Dataset.from_pandas(ds)
data_set

Dataset({
    features: ['Unnamed: 0', 'question', 'response', 'correct_answer'],
    num_rows: 150
})

In [5]:
feedback_dataset = rg.FeedbackDataset(
    guidelines="Grade the retrieval and generation results of the RAG chain",
    fields=[
        rg.TextField(name="query", title="User's question (query)"),
        rg.TextField(name="generated_response", title="Generated LLM response (answer) to the question"),
        rg.TextField(name="ground_truth", title="Ground Truth (Correct Answer)"),
    ],
    questions=[
        rg.LabelQuestion(
            name="correctness",
            title="Does the generated answer match the ground truth answer?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
        rg.LabelQuestion(
            name="usefulness",
            title="Is generated answer useful to resolve a question?",
            labels=["YES", "NO", "UNSURE"],
            required=True,
            visible_labels=None
            ),
    ]
)
feedback_dataset

FeedbackDataset(
   fields=[TextField(name='query', title="User's question (query)", required=True, type='text', use_markdown=False), TextField(name='generated_response', title='Generated LLM response (answer) to the question', required=True, type='text', use_markdown=False), TextField(name='ground_truth', title='Ground Truth (Correct Answer)', required=True, type='text', use_markdown=False)]
   questions=[LabelQuestion(name='correctness', title='Does the generated answer match the ground truth answer?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None), LabelQuestion(name='usefulness', title='Is generated answer useful to resolve a question?', description=None, required=True, type='label_selection', labels=['YES', 'NO', 'UNSURE'], visible_labels=None)]
   guidelines=Grade the retrieval and generation results of the RAG chain)
   metadata_properties=[])
   vectors_settings=[])
)

In [7]:
records = []
for i, item in enumerate(data_set):
    records.append(
        rg.FeedbackRecord(
            fields={
                "query": item["question"],
                "generated_response": item["response"],
                "ground_truth": item["correct_answer"],
            },
            external_id=f"record-{i}"
        )
    )

feedback_dataset.add_records(records)

In [8]:
remote_dataset = feedback_dataset.push_to_argilla(name="noRag-financebench150")

Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=4c74d7f6-2b4a-4303-86c9-e4bc1ef2f957
   name=noRag-financebench150
   workspace=Workspace(id=142a9c2d-f1dc-4704-8188-daf87f726164, name=admin, inserted_at=2024-07-03 09:01:38.246747, updated_at=2024-07-03 09:01:38.246747)
   url=https://winterforeststump-thesis.hf.space/dataset/4c74d7f6-2b4a-4303-86c9-e4bc1ef2f957/annotation-mode
   fields=[RemoteTextField(id=UUID('bba49e30-a1ab-4890-9031-5fd1059fe965'), client=None, name='query', title="User's question (query)", required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('f82090d3-01b4-488e-bdb5-4d218de310d1'), client=None, name='generated_response', title='Generated LLM response (answer) to the question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('c65e7fdd-4b36-4972-b118-e1350936813d'), client=None, name='ground_truth', titl

# Collect responses NO RAG Generaal QA

In [4]:
feedback = rg.FeedbackDataset.from_argilla("noRag-financebench150", workspace="admin")

In [5]:
list_values = []
for record_ix,record in enumerate(feedback):
  list_values.append({"record": str(record_ix+1),
                      "id": feedback.records[record_ix].id,
                      "query": feedback.records[record_ix].fields['query'],
                      "generated_response": feedback.records[record_ix].fields['generated_response'],
                      "usefulness_value": feedback.records[record_ix].responses[0].values["usefulness"].value,
                      "correctness_value": feedback.records[record_ix].responses[0].values["correctness"].value
                      })

In [6]:
len(list_values)

150

In [7]:
df = pd.DataFrame(list_values)
df.to_csv('eval_resuts_argilla_NoRagFinanceBench150.csv')